In [2]:
import math
import numpy as np
import torch.distributed as dist
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.utils.data
import yaml
from torch.cuda import amp
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import pandas as pd
import test  # import test.py to get mAP after each epoch
from models.yolo import Model
from utils.datasets import create_dataloader
from utils.general import (
    torch_distributed_zero_first, labels_to_class_weights, plot_labels, check_anchors, labels_to_image_weights,
    compute_loss, plot_images, fitness, strip_optimizer, plot_results, get_latest_run, check_dataset, check_file,
    check_git_status, check_img_size, increment_dir, print_mutation, plot_evolution, set_logging, init_seeds,
non_max_suppression)
from utils.google_utils import attempt_download
from utils.torch_utils import ModelEMA, select_device, intersect_dicts
from utils.torch_utils import select_device, time_synchronized
from models.experimental import attempt_load


class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

In [30]:
def geo_coords(box, x_translation, y_translation, geo_data, center=True):
  """
  Gets coordinates in epsg:4326 format (google maps) of bounding box
  If center, return only coordinate of center of bounding box
  Params:
    Box: [xmin, ymin, xmax, ymax]
  """
  transformer = Transformer.from_crs(geo_data['crs'], 'epsg:4326')
  if center:
    center_x, center_y = (box[2] + box[0])//2 + x_translation, (box[3] + box[1])//2 + y_translation
    new_coords = np.array(geo_data['transform']).reshape((3,3)) @ np.array( [center_x, center_y, 1])

    return transformer.transform(new_coords[0], new_coords[1])

  else:
    new_box = np.array([[x_translation + box[0], y_translation + box[1], 1],
            [x_translation + box[2], y_translation + box[3], 1]])
    new_coords = np.array(geo_data['transform']).reshape((3,3)) @ new_box.T

    return transformer.transform(new_coords[0], new_coords[1])

def get_polygon(xs, ys):
  xs, ys = sorted(xs), sorted(ys)
  return [[ys[0], xs[0]], [ys[1], xs[0]], [ys[1], xs[1]], [ys[0], xs[1]]]


In [32]:
import os
from shapely.geometry import Polygon
from pyproj import Transformer
import ast
df = pd.read_csv('../hunt_and_find/data_csv/new_test_250.csv', index_col=0).sort_values(['directory', 'file']).reset_index(drop=True)
def load_targets(annot_path, df_metadata, set_name='val'):
    coordinate_list = []
    for file in sorted(os.listdir(os.path.join(annot_path, set_name))):
        try:
            df_res = pd.read_csv(os.path.join(annot_path, set_name, file), header=None, sep=' ')
            num = int(file.replace('.txt', ''))
            md = df_metadata.loc[num]
            md['transform'] = ast.literal_eval(md['transform'].replace("'", '"'))
            for i, row in df_res.iterrows():
                width, height = row[2]* md['width'], row[3] * md['height']
                center_x, center_y = row[0] * md['width'], row[1] * md['height']
                box = [center_x- width//2, center_y - height//2, center_x + width // 2, center_y + width //2]
                xs, ys = geo_coords(box, md['x_pos'], md['y_pos'], md, center=False)
                coordinate_list.append(get_polygon(xs, ys))
        except Exception as e:
            pass
    return coordinate_list
coord_list = load_targets('../datasets/coolingTowers250/labels/', df)

/home/m.esguerra/.pyenv/versions/myenv/envs/finderenv/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [10]:
def experiment_evaluate(exp_path, thr=0.5):
    for file in os.listdir(os.path.join(exp_path, 'autolabels')):
        df_res = pd.read_csv(os.path.join(exp_path, 'autolabels', file), header=None)
            

,Unnamed: 0,asset,directory,file,tower_present,x_pos,y_pos,crop_number,transform,crs
0,5305,1rt0qailzbxl400stjw5pgfm3jtlqra7,datasets/batch2_250/,1rt0qailzbxl400stjw5pgfm3jtlqra7_crop_0000.npy,0,0,0,0,"[10.0, 0.0, 406200.0, 0.0, -10.0, 5686800.0, 0...",EPSG:32632
1,5500,1rt0qailzbxl400stjw5pgfm3jtlqra7,datasets/batch2_250/,1rt0qailzbxl400stjw5pgfm3jtlqra7_crop_0001.npy,0,220,0,1,"[10.0, 0.0, 406200.0, 0.0, -10.0, 5686800.0, 0...",EPSG:32632
2,1449,1rt0qailzbxl400stjw5pgfm3jtlqra7,datasets/batch2_250/,1rt0qailzbxl400stjw5pgfm3jtlqra7_crop_0007.npy,0,660,220,7,"[10.0, 0.0, 406200.0, 0.0, -10.0, 5686800.0, 0...",EPSG:32632
3,2663,1rt0qailzbxl400stjw5pgfm3jtlqra7,datasets/batch2_250/,1rt0qailzbxl400stjw5pgfm3jtlqra7_crop_0008.npy,0,0,440,8,"[10.0, 0.0, 406200.0, 0.0, -10.0, 5686800.0, 0...",EPSG:32632
4,2548,1rt0qailzbxl400stjw5pgfm3jtlqra7,datasets/batch2_250/,1rt0qailzbxl400stjw5pgfm3jtlqra7_crop_0009.npy,1,220,440,9,"[10.0, 0.0, 406200.0, 0.0, -10.0, 5686800.0, 0...",EPSG:32632
...,...,...,...,...,...,...,...,...,...,...
2437,18426,x1gvlsrn84duqeksn5cxxu81iaw32ufc,datasets/platform_250/,x1gvlsrn84duqeksn5cxxu81iaw32ufc_crop_0014.npy,0,440,880,14,"[10.0, 0.0, 451560.0, 0.0, -10.0, 5715420.0, 0...",EPSG:32633
2438,18355,yc9jxiya62tykohfsit0335exjch5r9u,datasets/platform_250/,yc9jxiya62tykohfsit0335exjch5r9u_crop_0005.npy,0,440,220,5,"[10.0, 0.0, 496500.0, 0.0, -10.0, 5809140.0, 0...",EPSG:32632
2439,18228,yc9jxiya62tykohfsit0335exjch5r9u,datasets/platform_250/,yc9jxiya62tykohfsit0335exjch5r9u_crop_0007.npy,1,220,440,7,"[10.0, 0.0, 496500.0, 0.0, -10.0, 5809140.0, 0...",EPSG:32632
2440,18388,za950bkry6n5hd6pzld3uh0bjc7d70d8,datasets/platform_250/,za950bkry6n5hd6pzld3uh0bjc7d70d8_crop_0012.npy,2,440,440,12,"[10.0, 0.0, 746940.0, 0.0, -10.0, 2452020.0, 0...",EPSG:32644


In [ ]:
def

In [22]:
import argparse
import os
import shutil
import time
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random

from utils.datasets import LoadStreams, LoadImages
from utils.general import (
    check_img_size, non_max_suppression, apply_classifier, scale_coords,
    xyxy2xywh, plot_one_box, strip_optimizer, set_logging)
from utils.torch_utils import select_device, load_classifier, time_synchronized
